In [ ]:
#!/usr/bin/python
# coding: utf-8

#################################################"    
##                 MODULES                  #####"
#################################################"
from scipy.signal import detrend
import matplotlib.pyplot as plt 
import matplotlib.dates as dt
from datetime import datetime 
import scipy.io as scio
import pandas as pd
import numpy as np
import os as os

 

#import pylab as py

        
###################################################################################   
class NKE:
###################################################################################   
###############################    
###############################
    def _load_wisens(self):
        """ Load a ascii file from a wisens NKE"""
        [adr, ext]=os.path.splitext(os.path.basename(self.filenameNKE))
        Date=[]

        # open
        rawdata=pd.read_csv(self.filenameNKE, skiprows=1,sep='\t',names=['DateTime', 'Temp', 'Pressure'])
        
        # Datetime manipulation
        dateheure=rawdata.DateTime.values
        for l in dateheure:
            temp=(datetime.strptime(l,'%Y-%m-%d %H:%M:%S.%f'))
            T=dt.date2num(temp)
            Date.append(T)
        
        #Output        
        self.Date=np.array(Date)
        self.Temperature=np.array(rawdata.Temp)      
        self.Pressure=np.array(rawdata.Pressure)

        print ('-----------------------------------------------')
        print ('Lecture ', adr, '............OK')
        print ('-----------------------------------------------')

################################
##    def fun_params(self, filename):
##        """ load params.txt """
##        print('----------------------------------------------')
##        print('Lecture params.txt :')
##        paramsdict={}
##        with open(filename) as f:
##        # build a dict with all the params file
##            for line in f:
##                if not line.startswith('#'):
##                    line=line.replace('#', '=')
##                    paramsline=filter(None, line.strip().split("="))
##                    paramsdict[paramsline[0]]=paramsline[1]
##        # Modif of variables in dict
##        listevariable=[]
##        for texte in paramsdict.keys():
##            listevariable=py.append(listevariable,texte)
##        for i,j in paramsdict.items():
##            for k in listevariable:
##                if i.find(k)!=-1:                    
##                    exec('self.'+i+'='+j.strip())
##                    message='{:<18}{:.<20}{:.>5}'.format( k,j.strip(), 'Ok')
##                    print(message)
##        print('----------------------------------------------')
        
##################################################################################   
    def _loadmat(self):
        """ Load a matlab file from previous run"""
        # initialisation
        variable=[]
        output=[]
        # Load mat file and put variable in output
        fmat=scio.loadmat(self.filenameNKE,squeeze_me=True)
        print ('---------------------------------------------------------------')
        print ('Load matlab file: ')
        print (self.filenameNKE)
        print ('---------------------------------------------------------------')
        for var in fmat.keys():
            variable=fmat[var]
            getattr(output,'extend')([variable])
            exec('self.'+var+ '=variable')
            message='{:<10}{:.<30}{:.>40}'.format('Variable ', var, 'Ok')
            print(message)
        print ('---------------------------------------------------------------')

##################################################################################  

                                                                      #      
                                                                     ###                  
                                                                    # # #
                                                                      #
                                                                      #
##########################################################################################################################################################  
                                                                      #
                                                                      #
                                                                    # # #
                                                                     ###         
                                                                      #
          
##########################################################"    
##                 PYTHON SCRIPT                     #####"
##########################################################"


#############################
### DEF VARIABLES
############################# 
data_wisens=NKE()

##########################################################  
###              OPEN FILE                             ###
##########################################################  

data_wisens.filenameNKE='4292_data_20190118_095748.mat'
# Load file
[adr, ext]=os.path.splitext(os.path.basename(data_wisens.filenameNKE))
if ext=='.mat':
    # READ .MAT FILE: FOR WORK IN PROGRESS
    data_wisens._loadmat()
else:
    # READ .TXT FILE
    data_wisens._load_wisens() # Pressure in dbar

##########################################################  
###              PROCESS                               ###
##########################################################  

# Fonction
u=data_wisens.Pressure[9600:12000]
t=data_wisens.Date[9600:12000]

# Parameters
N=len(u)
Te=data_wisens.Date[1]-data_wisens.Date[0]

# Process ?
    
#Analyse statistique
nbrv=(np.diff(np.sign(u)) < 0).sum()# Nbr d'oscillations
hist, bin_edges = np.histogram(u, bins=50, density=True) #Fonction de repartition des hauteurs de vagues. 
X = np.sort(u)
F = np.array(range(N))*100/float(N)
H13=np.mean(X[np.where(F>66)])      # Calcul de la moyenne du tiers superieur des oscillations
fu=np.where((np.diff(np.sign(u)) < 0))
Tm=np.mean(np.diff(fu))*Te    # Calcul de la periode moyenne

# Figure
fig1=plt.figure(figsize=(10,5))
plt.plot(t, u)
plt.xlabel("Date")
plt.ylabel("Amplitude")
plt.grid()
date_format = dt.DateFormatter('%d/%m %H:%M')
plt.gca().xaxis.set_major_formatter(date_format)
# Figure
fig2=plt.figure(figsize=(10,5))
plt.bar(bin_edges[:-1], hist, width=bin_edges[1]-bin_edges[0], color='red', alpha=0.5, label='Histogramm')
plt.plot(X, F, label=u'cumulative frequency')
plt.xlabel("oscillation amplitude")
plt.ylabel(u"Probability")
titre='Amplitude histogram: H1/3=' + str(np.around(H13,2))+'; Tm='+str(np.around(Tm,2))
plt.title(titre)
plt.legend()
plt.grid()
plt.ylim(0,100)
plt.show()

                

                                                                      #      
                                                                     ###                  
                                                                    # # #
                                                                      #
                                                                      #
##########################################################################################################################################################